# CISC 351 - Advanced Data Analytics
## Term Project (Winter 2019)
### Feature Engineering Notebook
### Home Credit Default Risk
### Mike Cruickshank


Data set is from the Home Credit Default Risk prediction competition on Kaggle
* https://www.kaggle.com/c/home-credit-default-risk

In [203]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import featuretools as ft

### Import Data

In [249]:
rows = 10000
app_test = pd.read_csv("application_test.csv", nrows = rows)
app_train = pd.read_csv("application_train.csv",nrows = rows)
bureau = pd.read_csv("bureau.csv", nrows = rows)
bureau_balance = pd.read_csv("bureau_balance.csv", nrows = rows)
credit = pd.read_csv("credit_card_balance.csv", nrows = rows)
installments = pd.read_csv("installments_payments.csv", nrows = rows)
cash = pd.read_csv("POS_CASH_balance.csv", nrows = rows)
previous = pd.read_csv("previous_application.csv", nrows = rows)
#sample_submission = pd.read_csv("sample_submission.csv", nrows = 100)

In [250]:
app_train.head()
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [251]:
app_test['TARGET'] = np.nan

app = app_train.append(app_test, ignore_index = True, sort = True)

In [252]:
for index in ['SK_ID_CURR', 'SK_ID_PREV','SK_ID_BUREAU']:
    for dataset in [app, bureau_balance, cash, credit, previous, installments]:
        if index in list(dataset.columns):
            dataset[index] = dataset[index].fillna(0).astype(np.int64)

### Feature Tools

In [253]:
es = ft.EntitySet(id = 'clients')

#### Variable Types

In [254]:
import featuretools.variable_types as vtypes

In [255]:
app_types = {}

for col in app:
    if(app[col].nunique() == 2) and (app[col].dtype == float):
        app_types[col] = vtypes.Boolean
        
del app_types['TARGET']

print('There are {} Boolean variables in the application data.'.format(len(app_types)))

There are 0 Boolean variables in the application data.


In [256]:
app_types['REGION_RATING_CLIENT'] = vtypes.Ordinal 
app_types['REGION_RATING_CLIENT_W_CITY'] = vtypes.Ordinal 
app_types['HOUR_APPR_PROCESS_START'] = vtypes.Ordinal 

In [257]:
previous_types = {}
for col in previous:
    if (previous[col].nunique() == 2) and (previous[col].dtype == float):
        previous_types[col] = vtypes.Boolean

print('There are {} Boolean variables in previous data.'.format(len(previous_types)))


There are 1 Boolean variables in previous data.


In [258]:
installments = installments.drop(columns = ['SK_ID_CURR'])
credit = credit.drop(columns = ['SK_ID_CURR'])
cash = cash.drop(columns = ['SK_ID_CURR'])

#### Adding Entities

In [259]:
es = es.entity_from_dataframe(entity_id = 'app', dataframe = app, index = 'SK_ID_CURR',
                             variable_types = app_types)
es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, index = 'SK_ID_BUREAU')

es = es.entity_from_dataframe(entity_id = 'previous', dataframe = previous, index = 'SK_ID_PREV',
                              variable_types = previous_types)

# Entities that do not have a unique index
es = es.entity_from_dataframe(entity_id = 'bureau_balance', dataframe = bureau_balance, 
                              make_index = True, index = 'bureaubalance_index')

es = es.entity_from_dataframe(entity_id = 'cash', dataframe = cash, 
                              make_index = True, index = 'cash_index')

es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installments,
                              make_index = True, index = 'installments_index')

es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit,
                              make_index = True, index = 'credit_index')

In [260]:
es

Entityset: clients
  Entities:
    app [Rows: 20000, Columns: 122]
    bureau [Rows: 10000, Columns: 17]
    previous [Rows: 10000, Columns: 37]
    bureau_balance [Rows: 10000, Columns: 4]
    cash [Rows: 10000, Columns: 8]
    installments [Rows: 10000, Columns: 8]
    credit [Rows: 10000, Columns: 23]
  Relationships:
    No relationships

#### Adding Relationships

In [261]:
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'], es['bureau']['SK_ID_CURR'])
r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])

r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'], es['previous']['SK_ID_CURR'])
r_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'], es['credit']['SK_ID_PREV'])

In [262]:
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])

In [263]:
es

Entityset: clients
  Entities:
    app [Rows: 20000, Columns: 122]
    bureau [Rows: 10000, Columns: 17]
    previous [Rows: 10000, Columns: 37]
    bureau_balance [Rows: 10000, Columns: 4]
    cash [Rows: 10000, Columns: 8]
    installments [Rows: 10000, Columns: 8]
    credit [Rows: 10000, Columns: 23]
  Relationships:
    bureau.SK_ID_CURR -> app.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV

#### Feature Primitives

In [264]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(5)


,name,type,description
0,skew,aggregation,Time since last related instance.
1,sum,aggregation,Time since last related instance.
2,min,aggregation,Time since last related instance.
3,count,aggregation,Time since last related instance.
4,num_true,aggregation,Time since last related instance.


In [265]:
primitives[primitives['type'] == 'transform'].head(5)


,name,type,description
19,numwords,transform,
20,hour,transform,
21,less_than_scalar,transform,
22,less_than,transform,
23,and,transform,


#### Deep Feature Synthesis 

In [266]:
default_agg_primitives = ['sum','std','max','skew','min','mean','count','percent_true','num_unique','mode']
default_trans_primitives = ['day','year','month','weekday','haversine','numwords','characters']

feature_names = ft.dfs(entityset = es, target_entity = 'app', 
                      trans_primitives = default_trans_primitives, 
                      agg_primitives = default_agg_primitives,
                      where_primitives = [], seed_features = [],
                      max_depth = 2, n_jobs = -1, verbose = 1,
                      features_only = True)

Built 1580 features


In [267]:
feature_names[-15:]

[<Feature: MEAN(previous.MEAN(credit.AMT_RECIVABLE))>,
 <Feature: MEAN(previous.MEAN(credit.AMT_TOTAL_RECEIVABLE))>,
 <Feature: MEAN(previous.MEAN(credit.CNT_DRAWINGS_ATM_CURRENT))>,
 <Feature: MEAN(previous.MEAN(credit.CNT_DRAWINGS_CURRENT))>,
 <Feature: MEAN(previous.MEAN(credit.CNT_DRAWINGS_OTHER_CURRENT))>,
 <Feature: MEAN(previous.MEAN(credit.CNT_DRAWINGS_POS_CURRENT))>,
 <Feature: MEAN(previous.MEAN(credit.CNT_INSTALMENT_MATURE_CUM))>,
 <Feature: MEAN(previous.MEAN(credit.SK_DPD))>,
 <Feature: MEAN(previous.MEAN(credit.SK_DPD_DEF))>,
 <Feature: MEAN(previous.COUNT(credit))>,
 <Feature: MEAN(previous.NUM_UNIQUE(credit.NAME_CONTRACT_STATUS))>,
 <Feature: NUM_UNIQUE(previous.MODE(cash.NAME_CONTRACT_STATUS))>,
 <Feature: NUM_UNIQUE(previous.MODE(credit.NAME_CONTRACT_STATUS))>,
 <Feature: MODE(previous.MODE(cash.NAME_CONTRACT_STATUS))>,
 <Feature: MODE(previous.MODE(credit.NAME_CONTRACT_STATUS))>]

#### Selecting Primitives

In [280]:
agg_primitives = ['sum']
trans_primitives = ['day']
feature_names = ft.dfs(entityset=es, target_entity='app',
                      agg_primitives = agg_primitives, 
                      trans_primitives = trans_primitives,
                      n_jobs = -1, verbose = 1,
                      features_only = True, 
                      max_depth = 2)

Built 183 features


In [290]:
from featuretools.primitives import make_agg_primitive, make_trans_primitive

def absolute(column):
    return abs(column)

Absolute = make_trans_primitive(function=absolute,
                               input_types=[vtypes.Numeric],
                               return_type=vtypes.Numeric)

def maximum(column): 
    return max(column)

Maximum = make_agg_primitive(function=maximum, input_types=[vtypes.Numeric],
                            return_type=vtypes.Numeric)

def num_over_1000(column):
    count = 0
    for i in range(len(column)):
        if (column.iloc[i] > 1000):
            count = count + 1
    return count

NumOver1000 = make_agg_primitive(function=num_over_1000, input_types = [vtypes.Numeric],
                                return_type = vtypes.Numeric)
    

#### Run Full Deep Feature Synthesis

In [278]:
import sys
print('Total size of entityset: {:.5f} gb.'.format(sys.getsizeof(es)/1e9))

Total size of entityset: 0.05982 gb.


In [279]:
import psutil

print('Total number of cpus detected: {}.'.format(psutil.cpu_count()))
print('Total size of system memory: {:.5} gb.'.format(psutil.virtual_memory().total/1e9))

Total number of cpus detected: 4.
Total size of system memory: 12.745 gb.


In [281]:
feature_matrix, feature_names = ft.dfs(entityset=es, target_entity = 'app',
                                      agg_primitives = agg_primitives, 
                                      trans_primitives = trans_primitives,
                                      n_jobs = 1, verbose = 1, features_only = False, 
                                      max_depth = 2, chunk_size = 500)

Built 183 features

Elapsed: 00:00 | Remaining: ? | Progress:   0%|          | Calculated: 0/40 chunks
Elapsed: 00:00 | Remaining: 00:15 | Progress:   2%|▎         | Calculated: 1/40 chunks
Elapsed: 00:00 | Remaining: 00:14 | Progress:   5%|▌         | Calculated: 2/40 chunks
Elapsed: 00:01 | Remaining: 00:14 | Progress:   8%|▊         | Calculated: 3/40 chunks
Elapsed: 00:01 | Remaining: 00:13 | Progress:  10%|█         | Calculated: 4/40 chunks
Elapsed: 00:01 | Remaining: 00:14 | Progress:  12%|█▎        | Calculated: 5/40 chunks
Elapsed: 00:02 | Remaining: 00:14 | Progress:  15%|█▌        | Calculated: 6/40 chunks
Elapsed: 00:02 | Remaining: 00:14 | Progress:  18%|█▊        | Calculated: 7/40 chunks
Elapsed: 00:03 | Remaining: 00:13 | Progress:  20%|██        | Calculated: 8/40 chunks
Elapsed: 00:03 | Remaining: 00:13 | Progress:  22%|██▎       | Calculated: 9/40 chunks
Elapsed: 00:04 | Remaining: 00:12 | Progress:  25%|██▌       | Calculated: 10/40 chunks
Elapsed: 00:04 | Remaining

In [291]:
feature_matrix, feature_names = ft.dfs(entityset=es, target_entity = 'app',
                                      agg_primitives = [NumOver1000], 
                                      trans_primitives = [Absolute],
                                      n_jobs = 1, verbose = 1, features_only = False, 
                                      max_depth = 2, chunk_size = 500)

Built 409 features



Elapsed: 00:00 | Remaining: ? | Progress:   0%|          | Calculated: 0/40 chunks


Elapsed: 00:01 | Remaining: 00:48 | Progress:   2%|▎         | Calculated: 1/40 chunks


Elapsed: 00:02 | Remaining: 00:46 | Progress:   5%|▌         | Calculated: 2/40 chunks


Elapsed: 00:03 | Remaining: 00:46 | Progress:   8%|▊         | Calculated: 3/40 chunks


Elapsed: 00:04 | Remaining: 00:44 | Progress:  10%|█         | Calculated: 4/40 chunks


Elapsed: 00:06 | Remaining: 00:44 | Progress:  12%|█▎        | Calculated: 5/40 chunks


Elapsed: 00:07 | Remaining: 00:41 | Progress:  15%|█▌        | Calculated: 6/40 chunks


Elapsed: 00:08 | Remaining: 00:38 | Progress:  18%|█▊        | Calculated: 7/40 chunks


Elapsed: 00:09 | Remaining: 00:36 | Progress:  20%|██        | Calculated: 8/40 chunks


Elapsed: 00:10 | Remaining: 00:35 | Progress:  22%|██▎       | Calculated: 9/40 chunks


Elapsed: 00:11 | Remaining: 00:34 | Progress:  25%|██▌       | Calculated: 10/40 chunks


El

In [284]:
print(np.shape(feature_matrix))


(20000, 183)


In [292]:
feature_matrix.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,ABSOLUTE(NUM_OVER_1000(credit.AMT_RECEIVABLE_PRINCIPAL)),ABSOLUTE(NUM_OVER_1000(credit.AMT_RECIVABLE)),ABSOLUTE(NUM_OVER_1000(credit.AMT_TOTAL_RECEIVABLE)),ABSOLUTE(NUM_OVER_1000(credit.CNT_DRAWINGS_ATM_CURRENT)),ABSOLUTE(NUM_OVER_1000(credit.CNT_DRAWINGS_CURRENT)),ABSOLUTE(NUM_OVER_1000(credit.CNT_DRAWINGS_OTHER_CURRENT)),ABSOLUTE(NUM_OVER_1000(credit.CNT_DRAWINGS_POS_CURRENT)),ABSOLUTE(NUM_OVER_1000(credit.CNT_INSTALMENT_MATURE_CUM)),ABSOLUTE(NUM_OVER_1000(credit.SK_DPD)),ABSOLUTE(NUM_OVER_1000(credit.SK_DPD_DEF))
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,20560.5,568800.0,450000.0,135000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100003,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100004,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100005,17370.0,222768.0,180000.0,99000.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
